In [1]:
import sys
# This will print the path to the Python executable the kernel is using
print(sys.executable)

/home/nidhinninan/anaconda3/envs/CSCE5720/bin/python


In [2]:
!pip install -q transformers datasets torch accelerate huggingface_hub bitsandbytes

In [3]:
# HuggingFace authentication (for model download/upload)
from huggingface_hub import login
login()  # Will prompt for token - get from https://huggingface.co/settings/tokens

In [4]:
from datasets import load_dataset
import pandas as pd

# Load full dataset
full_dataset = load_dataset('derek-thomas/ScienceQA')

# Analyze dataset structure
print(f"Train samples: {len(full_dataset['train'])}")
print(f"Validation samples: {len(full_dataset['validation'])}")
print(f"Test samples: {len(full_dataset['test'])}")

# Examine sample
sample = full_dataset['train'][0]
print(sample.keys())

Train samples: 12726
Validation samples: 4241
Test samples: 4241
dict_keys(['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'])


In [5]:
# Explore structure
print(full_dataset['train'][1000])

{'image': None, 'question': 'Which sentence states a fact?', 'choices': ['Atlanta is the capital of Georgia.', 'Atlanta is too hot in the summer.'], 'answer': 0, 'hint': '', 'task': 'closed choice', 'grade': 'grade5', 'subject': 'social science', 'topic': 'civics', 'category': 'Social studies skills', 'skill': 'Identify facts and opinions', 'lecture': 'A fact is something that can be proved to be true. Facts can be proved by observing, measuring, or studying information.\nThe flag of the United States has 13 stripes.\nThis is a fact. It can be proved by looking at the flag and counting the number of stripes.\nAn opinion is something that a person believes, thinks, or feels. An opinion cannot be proved true.\nThe flag of the United States is easy to draw.\nThis is an opinion. People may have different opinions about what makes a flag "easy" to draw.', 'solution': 'The second sentence states a fact.\nAtlanta is the capital of Georgia.\nIt can be proved by looking at a map of Georgia.\nTh

In [6]:
# Create balanced subset across topics
train_subset = full_dataset['train'].shuffle(seed=42).select(range(3000))
val_subset = full_dataset['validation'].shuffle(seed=42).select(range(500))
test_subset = full_dataset['test'].shuffle(seed=42).select(range(500))

# Save locally for reuse
train_subset.save_to_disk('data/train_subset')
val_subset.save_to_disk('data/val_subset')
test_subset.save_to_disk('data/test_subset')

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [7]:
def format_scienceqa_for_chat(example):
    """Convert ScienceQA to conversational format"""
    
    # Build question with choices
    question = example['question']
    choices = example['choices']
    choices_text = "\n".join([f"{chr(65+i)}. {choice}" for i, choice in enumerate(choices)])
    
    full_question = f"{question}\n\n{choices_text}"
    
    # Build answer with explanation
    answer_idx = example['answer']
    correct_answer = choices[answer_idx]
    
    response = f"The correct answer is {chr(65+answer_idx)}. {correct_answer}"
    
    # Add explanation if available
    if example.get('solution'):
        response += f"\n\nExplanation: {example['solution']}"
    
    # Add lecture context if available
    if example.get('lecture'):
        response += f"\n\nBackground: {example['lecture']}"
    
    # Format as conversational message
    return {
        "messages": [
            {"role": "system", "content": "You are a helpful science tutor for elementary through high school students. Explain concepts clearly with examples."},
            {"role": "user", "content": full_question},
            {"role": "assistant", "content": response}
        ]
    }

In [8]:
# Apply formatting
train_formatted = train_subset.map(format_scienceqa_for_chat, remove_columns=train_subset.column_names)
val_formatted = val_subset.map(format_scienceqa_for_chat, remove_columns=val_subset.column_names)
test_formatted = test_subset.map(format_scienceqa_for_chat, remove_columns=test_subset.column_names)

In [9]:
# persist the formatted splits
train_formatted.save_to_disk("data/train_formatted")
val_formatted.save_to_disk("data/val_formatted")
test_formatted.save_to_disk("data/test_formatted")

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]